In [1]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("Jupyter") \
    .config("spark.master", "spark://spark-master:7077") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.cores", "2") \
    .config("spark.driver.memory", "2g") \
    .config("spark.cores.max", "4") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .enableHiveSupport()

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark

:: loading settings :: url = jar:file:/usr/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5764e4c6-29cb-430b-8bf2-6fb5f9f241eb;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/2.4.0/delta-core_2.12-2.4.0.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;2.4.0!delta-core_2.12.jar (1323ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/2.4.0/delta-storage-2.4.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;2.4.0!delta-storage.jar (261ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (279ms)
:: resolution report :: resolve 3501ms :: artifacts dl 1868m

## Testando Conexão HDFS

In [2]:
#Listando diretorios/arquivos
!hdfs dfs -ls /users/

Found 3 items
drwxr-xr-x   - Daniel root                0 2024-08-10 20:16 /users/Daniel
drwxr-xr-x   - root   supergroup          0 2024-08-13 03:29 /users/Test
drwxr-xr-x   - root   supergroup          0 2024-08-10 20:21 /users/hive


In [5]:
#Criando diretorio
!hdfs dfs -rmdir /users/Teste

In [3]:
#movendo arquivos do local para o HDFS
!hdfs dfs -put /usr/notebooks/curva.csv /users/Teste

put: `/users/Test/curva.csv': File exists


## Testando Hive

In [7]:
spark.sql('create database teste')

DataFrame[]

In [8]:
spark.sql('show databases').show()

+---------+
|namespace|
+---------+
|  default|
|    teste|
+---------+



## Testando Spark

In [2]:
spark.range(10).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [10]:
#Lendo arquivos do HDFS
df = spark.read.csv('/users/Teste/curva.csv',header=True,sep=';')

df.show()

+-----+-------+
|curva|funding|
+-----+-------+
| CC01|   0.75|
| CC02|   0.65|
| CC03|   0.75|
| CC04|   0.65|
| CC05|   0.75|
| CC06|   0.65|
+-----+-------+



In [11]:
#salvando tabela delta
df.write.format("delta").mode("overwrite").saveAsTable('teste.curva')

24/08/13 04:02:25 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/08/13 04:02:32 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`teste`.`curva` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


In [12]:
#Verificando resultados
spark.sql("select * from teste.curva").show()

+-----+-------+
|curva|funding|
+-----+-------+
| CC01|   0.75|
| CC02|   0.65|
| CC03|   0.75|
| CC04|   0.65|
| CC05|   0.75|
| CC06|   0.65|
+-----+-------+

